# YOLOv8 Training on Google Colab

Bu notebook, YOLOv8 modelini Google Colab üzerinde eğitmek için hazirlanmistir.

**Hazirlik:**
1. `merged_dataset.zip` dosyasini Colab'in sol menüsündeki dosya bölümüne sürükleyip birakin (veya Drive'dan çekin).
2. `merged_data.yaml` dosyasini da ayni sekilde yükleyin.
3. `Runtime` menüsünden `Change runtime type` diyerek **T4 GPU** (veya daha iyisini) seçtiginizden emin olun.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/YOLO?PLAKA/colab


/content/drive/MyDrive/YOLO?PLAKA/colab


In [3]:
!ls

Colab_Train.ipynb  merged_dataset.zip  runs	   yolov8l.pt
merged_dataset	   merged_data.yaml    yolo11n.pt


In [4]:
# 1. Gerekli Kütüphanelerin Kurulumu
!pip install ultralytics

In [5]:
!ls

Colab_Train.ipynb  merged_dataset.zip  runs	   yolov8l.pt
merged_dataset	   merged_data.yaml    yolo11n.pt


In [ ]:
# 2. Veri Setinin Hazirlanmasi (Zip'ten Çikarma)
import os
import zipfile

# Eger zip dosyasi varsa açalim
if os.path.exists("merged_dataset.zip"):
    print("Zip dosyasi bulundu, çikartiliyor...")
    with zipfile.ZipFile("merged_dataset.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    print("Çikartma tamamlandi: ./merged_dataset")
else:
    print("UYARI: 'merged_dataset.zip' bulunamadi. Lütfen dosyalari yüklediginizden emin olun.")

Zip dosyasi bulundu, çikartiliyor...
Çikartma tamamlandi: ./merged_dataset


In [6]:
!ls merged_dataset/train
!ls merged_dataset/train/labels


Streaming output truncated to the last 5000 lines.
CarLongPlateGen2419_jpg.rf.799889e969a936b115f2f15d44146961.txt
CarLongPlateGen241_jpg.rf.b0b44e7e2d86c5bd3180636ab4c16a1d.txt
CarLongPlateGen2420_jpg.rf.83442f1d92971c62c71cbbab58fc8895.txt
CarLongPlateGen2421_jpg.rf.7b36bb0036c99a3440c72bb5a6c4bb42.txt
CarLongPlateGen2423_jpg.rf.69c1cde1686822c92969c48ffe819672.txt
CarLongPlateGen2424_jpg.rf.cc2b12aeabe8da20b9e61486d359601e.txt
CarLongPlateGen2425_jpg.rf.d5fafe80e0175983f25b29d569069fc6.txt
CarLongPlateGen2426_jpg.rf.a24b07e7fe39314423a86143f421efd2.txt
CarLongPlateGen2427_jpg.rf.c5957fb4e8dda129451a09bc0a58bfea.txt
CarLongPlateGen2428_jpg.rf.6265f044945d48757fa20b3f57053cd3.txt
CarLongPlateGen2429_jpg.rf.0c190b3429ef0a71e96a5ec5cc0eec2c.txt
CarLongPlateGen2430_jpg.rf.0198db3ad7bebdbbadb085d223717e08.txt
CarLongPlateGen2431_jpg.rf.7ea2daddd97c81feee54d8d573d828b0.txt
CarLongPlateGen2432_jpg.rf.742c67977324439f6e8e39e0443b6c32.txt
CarLongPlateGen2433_jpg.rf.6080b9b71524969399597449293

In [8]:
# 3. merged_data.yaml Dosyasini Düzeltme (Colab Yollarina Göre)
# Colab'da çalisirken yollarin (path) dogru oldugundan emin olmaliyiz.
import os
import zipfile

import yaml

yaml_path = "merged_data.yaml"

if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    # Yollari Colab'in kök dizinine (content) göre ayarliyoruz
    data['path'] = './merged_dataset'
    data['train'] = 'train/images'
    data['val'] = 'valid/images'
    data['test'] = 'test/images'

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    print("merged_data.yaml Colab için güncellendi:")
    print(data)
else:
    print("HATA: merged_data.yaml dosyasi bulunamadi!")

merged_data.yaml Colab için güncellendi:
{'names': ['License_Plate'], 'nc': 1, 'path': './merged_dataset', 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}


In [7]:
!ls

Colab_Train.ipynb  merged_dataset.zip  runs	   yolov8l.pt
merged_dataset	   merged_data.yaml    yolo11n.pt


In [9]:
# 4. Egitimi Baslatma (YOLOv8 Large)
from ultralytics import YOLO

# Modeli yükle (yolov8l.pt otomatik indirilecek)
print("Model yükleniyor: YOLOv8l...")
model = YOLO("yolov8l.pt")

# Egitimi baslat
print("Egitim basliyor...")
results = model.train(
    data="merged_data.yaml",
    epochs=50,
    imgsz=640,
    batch=16, # Colab T4 GPU için genelde 16 uygundur, hata verirse 8 yapin
    name="colab_train_large",
    device=0
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model yükleniyor: YOLOv8l...
Egitim basliyor...
Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=merged_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz

In [12]:
# 5. Sonuçlari Zipleyip İndirme Hazirligi
!zip -r ./training_results.zip ./runs/detect/colab_train_large8

print("Egitim sonuçlari 'training_results.zip' olarak hazirlandi. Sol menüden dosyayi indirip 'best.pt'yi alabilirsiniz.")

# Colab dosya indirme kodu (Google Chrome'da otomatik açilabilir)
from google.colab import files
try:
    files.download("./training_results.zip")
except:
    print("Otomatik indirme baslamadi. Sol menüden 'training_results.zip' dosyasini manuel indirebilirsiniz.")

  adding: runs/detect/colab_train_large8/ (stored 0%)
  adding: runs/detect/colab_train_large8/weights/ (stored 0%)
  adding: runs/detect/colab_train_large8/weights/last.pt (deflated 8%)
  adding: runs/detect/colab_train_large8/weights/best.pt (deflated 8%)
  adding: runs/detect/colab_train_large8/args.yaml (deflated 51%)
  adding: runs/detect/colab_train_large8/labels.jpg (deflated 42%)
  adding: runs/detect/colab_train_large8/train_batch0.jpg (deflated 6%)
  adding: runs/detect/colab_train_large8/train_batch1.jpg (deflated 8%)
  adding: runs/detect/colab_train_large8/train_batch2.jpg (deflated 4%)
  adding: runs/detect/colab_train_large8/results.csv (deflated 61%)
  adding: runs/detect/colab_train_large8/train_batch18480.jpg (deflated 13%)
  adding: runs/detect/colab_train_large8/train_batch18481.jpg (deflated 12%)
  adding: runs/detect/colab_train_large8/train_batch18482.jpg (deflated 13%)
  adding: runs/detect/colab_train_large8/val_batch0_labels.jpg (deflated 10%)
  adding: runs/d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>